In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil
from open3d.pipelines import registration
from predict_maskpls import RefinerModel

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/cedric/anaconda3/envs/torch/lib/python3.9/site-packages/MinkowskiEngine-0.5.4-py3.9-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
SEQUENCE_NUM = 7

ind_start = 0
ind_end = 1100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'


dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

This cell can be ignored after first run as outputs are stored 

Now we subsample the poses based on a voxel_size

In [3]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

In [4]:
alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0





out_kitti_instance = out_chunks + 'out_kitti_instance2/'

        
out_refined = out_chunks + 'out_refined/'
if os.path.exists(out_refined) == True : 
        shutil.rmtree(out_refined)
os.makedirs(out_refined)


maskpls = RefinerModel()
eval_refiner = True 

out_data = []

center_ids = os.listdir(out_kitti_instance)
for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                
                name = center_ids[sequence]
                print("sequence",sequence)
                cur_pcd = o3d.io.read_point_cloud(out_kitti_instance + name)
                predicted_refined = maskpls.forward_and_project(cur_pcd)
                #maskpls.project_pseudo_labels(cur_pcd,center_ids[sequence],name)
                o3d.io.write_point_cloud(out_refined + name, predicted_refined,write_ascii=False, compressed=False, print_progress=False)
                
                        #pcd_dbscan = DBSCAN_clustering_logic(cur_pcd,
                        #                        eps=0.3, min_samples=10)
                        #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                
                
                #kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,kitti_labels['ground']['instance'][sequence].reshape(-1,))
                #o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                

        
        
        

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds


def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels

def remove_semantics_pcd(pcd,labels,preds):
        cols = np.asarray(pcd.colors)
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
                        cols[pred_idcs] = np.array([0,0,0])
                        
        pcd.colors = o3d.utility.Vector3dVector(cols)   
        return pcd




In [ ]:
out_kitti = out_chunks + 'out_kitti/'
out_kitti_instance = out_chunks + 'out_kitti_instance/'


#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)[:-1]
point_clouds_refined = get_merge_pcds(out_refined)[:-1]

#merge = merge_chunks_unite_instances(point_clouds)
#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
#merge_kitti = merge_unite_gt(point_clouds_kitti)
#merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
merge_refined = merge_chunks_unite_instances(point_clouds_refined)

#o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_refined.pcd", merge_refined, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_dbscan.pcd", merge_dbscan, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti.pcd", merge_kitti, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance.pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)

#unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)



[]
['000066.pcd', '000093.pcd', '000125.pcd', '000164.pcd', '000193.pcd', '000220.pcd', '000249.pcd', '000278.pcd', '000336.pcd', '000368.pcd', '000391.pcd']
['000066.pcd', '000093.pcd', '000125.pcd', '000164.pcd', '000193.pcd', '000220.pcd', '000249.pcd', '000278.pcd', '000336.pcd', '000368.pcd', '000391.pcd', '000415.pcd', '000441.pcd', '000483.pcd', '000515.pcd', '000540.pcd', '000565.pcd', '000587.pcd', '000610.pcd', '000650.pcd', '000774.pcd', '000794.pcd', '000813.pcd', '000833.pcd', '000854.pcd', '000877.pcd', '000915.pcd', '000949.pcd', '000982.pcd', '001048.pcd']


True

In [ ]:
'''
optional cell for loading stored files
import open3d as o3d

out_folder = 'pcd_preprocessed/'
merge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')
merge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')
merge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
merge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')

print(merge)
print(merge_kitti)
print(merge_exp2)
'''

"\noptional cell for loading stored files\nimport open3d as o3d\n\nout_folder = 'pcd_preprocessed/'\nmerge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')\nmerge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')\nmerge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')\nmerge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')\nmerge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')\n\nprint(merge)\nprint(merge_kitti)\nprint(merge_exp2)\n"

In [ ]:
#new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance7.pcd')
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined.pcd')
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

new_refined = remove_semantics_pcd(merge_refined,labels_kitti,labels_refined)

In [ ]:
unique_colors, labels_refined = np.unique(np.asarray(new_refined.colors),axis=0, return_inverse=True)
label_to_confidence = {}
pcd_cols = np.asarray(new_refined.colors) 
for label in list(np.unique(labels_refined)):
	idcs = np.where(labels_refined == label)[0]
	cur_color = pcd_cols[idcs[0]]
	key = str(int(cur_color[0] * 255)) + '|' + str(int(cur_color[1] * 255)) + '|' + str(int(cur_color[2] * 255))
	label_to_confidence[label] = maskpls.confs_dict[key]

In [ ]:
metrics_refined = Metrics(name='refined')


metrics_refined.update_stats(labels_refined,labels_kitti,label_to_confidence,calc_all=True)


Metrics for file refined
{'panoptic': 0.759014368240107, 'precision': 0.86, 'recall': 0.7962962962962963, 'fScore': 0.826923076923077, 'usr': 0.05, 'osr': 0.018518518518518517, 'noise': 0.09, 'missed': 0.12962962962962962, 'mean': 0.9178778406624549}
calc AP for overlap @0.25
Average Precision @ 0.25 0.8498736961043124
calc AP for overlap @0.5
Average Precision @ 0.5 0.7260979532907539
calc AP for overlap @0.55
Average Precision @ 0.55 0.6949233306985517
calc AP for overlap @0.6
Average Precision @ 0.6 0.6949233306985517
calc AP for overlap @0.65
Average Precision @ 0.65 0.6949233306985517
calc AP for overlap @0.7
Average Precision @ 0.7 0.5918976092733214
calc AP for overlap @0.75
Average Precision @ 0.75 0.5795877708458368
calc AP for overlap @0.8
Average Precision @ 0.8 0.5640699974851524
calc AP for overlap @0.85
Average Precision @ 0.85 0.5301729079190256
calc AP for overlap @0.9
Average Precision @ 0.9 0.4534579560214868
calc AP for overlap @0.95
Average Precision @ 0.95 0.295023

array([81, 81, 81, ...,  0,  0,  0])

In [ ]:
print(labels_refined.shape) 
print(labels_kitti.shape)

(19170491,)
(19170491,)


In [ ]:
o3d.visualization.draw_geometries([merge_refined])

In [ ]:
out_folder = 'pcd_preprocessed/'
merge_refined = o3d.io.read_point_cloud(out_folder + 'merge_refined.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
#unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)
#unique_colors, labels_refined = np.unique(np.asarray(merge_refined.colors),axis=0, return_inverse=True)

#new_refined = remove_semantics(labels_kitti,labels_refined)
#o3d.visualization.draw_geometries([color_pcd_by_labels(merge_refined,new_refined)])

[Open3D WARNING] Read PCD failed: unable to open file: pcd_preprocessed/merge_part_kitti_instance.pcd


In [ ]:
o3d.visualization.draw_geometries([merge_refined])

In [ ]:
fn = out_folder + "merge_refined.pcd"
fn2 = out_folder + 'ncuts_tarl_spatial1_all.pcd'
pcd = o3d.io.read_point_cloud(fn)
pcd_ncuts = o3d.io.read_point_cloud(fn2)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] Read PCD failed: unable to open file: pcd_preprocessed/ncuts_tarl_spatial1_all.pcd
